## Unificación de Maestros


In [2]:
import pandas as pd

import dependencies

import numpy as np

In [3]:
path_input = dependencies.rootFolder / 'data' / 'raw'
path_output = dependencies.rootFolder / 'data' / 'processed'

### Maestro de tipo de producto (tipo_producto)

Se considera el Maestro de la ETL previa, 

La información de tipo de producto se obtiene de la hoja "base datos 2022" del excel "Base de Datos Distribución Actual(MDI01-067) 2020.xlsx".

Se consideran las siguientes tres columnas: 

* Identificador (id)
* Tipo de producto (tipo_producto)
* Descripción (descripción)

Qué tener en cuenta: 

* Qué identificador se va a utlizar

In [4]:
df_tipo_producto = pd.read_excel(
    path_input / 'Base de Datos Distribución Actual (MDI01-067)  2020.xlsx',
    sheet_name='base  datos 2022',
    usecols='M', 
    names=['tipo_producto']
)
df_tipo_producto = df_tipo_producto.dropna(how='all')

## Se extraen valores unicos
df_tipo_producto= pd.DataFrame(pd.unique(df_tipo_producto["tipo_producto"]), columns=['tipo_producto'])

## Se añade la columna 'descripcion'
df_tipo_producto['descripcion'] = ['Canasta Basica', 'Producto Fresco', 'Extras']

## Tratanmiendo de columnas con datos tipo str 
df_tipo_producto["tipo_producto"] = df_tipo_producto["tipo_producto"].str.upper()
df_tipo_producto['tipo_producto'] = df_tipo_producto['tipo_producto'].str.lstrip()
df_tipo_producto['tipo_producto'] = df_tipo_producto['tipo_producto'].str.rstrip()

df_tipo_producto["descripcion"] = df_tipo_producto["descripcion"].str.title()
df_tipo_producto['descripcion'] = df_tipo_producto['descripcion'].str.lstrip()
df_tipo_producto['descripcion'] = df_tipo_producto['descripcion'].str.rstrip()

## Asignación de índices
df_tipo_producto.index += 1
df_tipo_producto = df_tipo_producto.reset_index(names="id_tipo_producto")
df_tipo_producto

,id_tipo_producto,tipo_producto,descripcion
0,1,CB,Canasta Basica
1,2,PF,Producto Fresco
2,3,E,Extras


### Maestro Conductor (conductor)

Se considera una muestra de la información histórica de los datos de sharepoint de ATTSF > 02. Datos > Bases de datos  desde al año 2019 

Se consideran unicamente dos columnas: 

* Identificador (id)
* conductor (conductor) / ¿puede ser nombre?

Qué tener en cuenta: 

* Qué identificador se va a utlizar
* Unificación de conductores 


In [5]:
## Path de archivos históricos 
path_input_historic = dependencies.rootFolder / 'data' / 'raw_historic'
path_output_historic = dependencies.rootFolder / 'data' / 'processed_historic'

Se toman datos del histórico de registros de distribuciones para hacer un count de los valores más frecuentes de los nombres de los conductores y así poder determinar cuales serían los que formarian parte del maestro. 

Se hacen las siguientes sutituciones de acuerdo con el valor de conductor que tiene más registros. 

- `Abdelmula` = Abbelmula, Abdeimula, Abdelmaula
- `Brahim Salem Salma` = Brahim Salma
- `Embarec Ahmed`= Embarek Ahmed
- `Lajlifa Moh. Fadel` = Lajlifa Moh.Fadel, Lalifa Moh. Fadel, ¿Lejlifa Mohamed Fadel? por lo pronto se deja en el maestro
- `Limam Moh. Mehdi` = Limam Moh-Mehdi
- `Mahfud Mohamed-Mehdi`= Mahfud Moh-Mehdi	
- `Mohamed Baba`= Mohames Baba,  ¿Mohamed Hama? por lo pornto se deja en el maestro 
- `Said Al-Lal` = Said Alal	
- `Said Nafi` = Saiid Nafi
- `Salma Mujtar` = Salma Mojtar, Selma Mujtar

Habrá que validar y sustituir estos datos en los registros históricos. 

In [6]:
## lista para almacenar los datos de cada archivo de Excel
data_list = []

# bucle para leer cada archivo de Excel en la carpeta y almacenar los datos en data_list
for file_name in os.listdir(path_input_historic):
    if file_name.endswith('.xlsx'): # comprobar si el archivo es un archivo de Excel
        data = pd.read_excel(
            path_input_historic / file_name,
            sheet_name = 'base  datos', ## optimizar búsqueda de hoja
            usecols='B',
            names='conductor', 
            header=2
        )
        data_list.append(data)

## combinar todos los datos en un solo DataFrame
df_datos_concat = pd.concat(data_list, axis=0, ignore_index=True).dropna(how='all')
df_datos_concat.columns = ['conductor']
df_conductor_concat = df_datos_concat.sort_values('conductor')

## Se tratan los datos para eliminar espaciós y poner el inicio del nombre con mayúscula 
df_conductor_concat['conductor'] = df_conductor_concat['conductor'].str.title()
df_conductor_concat['conductor'] = df_conductor_concat['conductor'].str.lstrip()
df_conductor_concat['conductor'] = df_conductor_concat['conductor'].str.rstrip()


## Referencia para crear el diccionario datos conductor
#df_conductor_count = pd.DataFrame(df_conductor_concat.value_counts('conductor'))
#df_conductor_count = df_conductor_count.sort_values('conductor')
#df_conductor_count

## se crea diccionario para unificación de nombres

d_unificados_conductor = {'Abdelmula': ['Abbelmula', 'Abdeimula', 'Abdelmaula'], 
                    'Brahim Salem Salma' : ['Brahim Salma'],
                    'Embarec Ahmed' : ['Embarek Ahmed'],
                    'Lajlifa Moh. Fadel' : ['Lajlifa Moh.Fadel', 'Lalifa Moh. Fadel', 'Lejlifa Mohamed Fadel'],  ## ¿Moh. se sustituye por Mohamed? en todos los casos 
                    'Limam Moh. Mehdi' : ['Limam Moh-Mehdi', 'Limam Moh. Mehadi'],
                    'Mahfud Mohamed-Mehdi': ['Mahfud Moh-Mehdi'],
                    'Mohamed Baba': ['Mohames Baba'],
                    'Said Al-Lal': ['Said Alal'],  ## ¿Se eliminarán puntos y guiones? 
                    'Said Nafi': ['Saiid Nafi'],
                    'Salma Mujtar': ['Salma Mojtar', 'Selma Mujtar'],
                    'Hassena Lehbib': ['Haswna Lehbib', 'Hasenna Lehbib'],
                    'Buel-La': ['Buel-lla', 'Buel-Lla'],
                    'Larbana Hamudi' : ['Iarbana Hamudi']
                    }

## Se guardan valores únicos 
df_conductor_concat = pd.DataFrame(data=df_conductor_concat['conductor'].unique())
df_conductor_concat.columns = ['conductor']
df_conductor_concat

## Bucle que unifica los datos 
for i in range(0, len(df_conductor_concat['conductor'])):
    for key in d_unificados_conductor.keys():
        for value in d_unificados_conductor[key]:
            if value in df_conductor_concat.loc[i, 'conductor']:
                df_conductor_concat.loc[i, 'conductor'] = df_conductor_concat.loc[i, 'conductor'].replace(value, key)

## Se crea y se guarda dataframe del maestro 
df_conductor = pd.DataFrame(df_conductor_concat['conductor'].unique())
df_conductor.columns = ['conductor']
df_conductor.index += 1
df_conductor = df_conductor.reset_index(names="id_conductor")
 
## se agrega nuevo conductor 
nuevo_conductor = pd.DataFrame({'id_conductor' : [24, 25, 26, 27], 'conductor': ['Hassena Lehbib', 'Brahim Hamdi', 'Mohamwd', 'Jalifa Mohamd']})
df_conductor = df_conductor.append(nuevo_conductor,  ignore_index=True)
df_conductor


C:\Users\MonicaAlcantarMartin\AppData\Local\Temp\ipykernel_1088\188850488.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_conductor = df_conductor.append(nuevo_conductor,  ignore_index=True)


,id_conductor,conductor
0,1,Abdelmula
1,2,Ahmed Salem
2,3,Beljeir
3,4,Bolla
4,5,Brahim Salem Salma
5,6,Buel-La
6,7,Chej Hamdeiti
7,8,Embarec Ahmed
8,9,Hasana Lehbib
9,10,Larbana Hamudi


### Maestro Wilaya (wilaya)

Se tienen en consideración tres orígenes de maestros 

1. El maestro de wilayas de ETL previo que se encuentra en Git en el directorio `app/data/processed/bbdd`.
2. El maestro de wilayas generado de la información histótica desde el año 2019, información optenida desde el sharepoint de ATTSF > 02. Datos > Bases de datos.

Se hacen las siguientes sustituciones de acuerdo con el nombre de Wilaya que se porporciona en el fichero `Talleres Regionales.kml`. 

- `Aaiun` = Aaiún
- `Bojador` = Bujador, Bujdur

- Los registro `Insituciones` y `Taller Central` quedan fuera de este maestro, se considerarán posteriormente para las tablas sobre las reparaciones en instalaciones. 

Habrá que validar y sustituir estos datos en los registros históricos. 

In [7]:
## Carga de datos de los maestros 
## maestro 1 
df_wilaya_1 = pd.read_csv(path_output / 'bbdd/wilaya.csv')
df_wilaya_1 = df_wilaya_1.drop('Unnamed: 0', axis=1)
df_wilaya_1.columns = ['wilaya']

## maestro 2 
df_wilaya_2 = pd.read_csv(path_output_historic / 'wilaya.csv')
df_wilaya_2 = df_wilaya_2.drop(['Unnamed: 0', 'id_wilaya'], axis=1)

## concatenar los maestros de origen
df_wilaya_concat = pd.concat([df_wilaya_1, df_wilaya_2], axis=0, ignore_index=True).sort_values('wilaya')
df_wilaya_aux = pd.DataFrame(df_wilaya_concat['wilaya'].unique(), columns=['wilaya'])
df_wilaya_aux

d_unificados_wilaya = {'Aaiun': ['Aaiún', 'Aiun'], 
                       'Bojador' : ['Bujador', 'Bujdur']
                      } 

## Bucle que unifica los datos 
for i in range(0, len(df_wilaya_aux['wilaya'])):
    for key in d_unificados_wilaya.keys():
        for value in d_unificados_wilaya[key]:
            if value in df_wilaya_aux.loc[i, 'wilaya']:
                df_wilaya_aux.loc[i, 'wilaya'] = df_wilaya_aux.loc[i, 'wilaya'].replace(value, key)



df_wilaya = pd.DataFrame(df_wilaya_aux['wilaya'].unique(), columns=['wilaya'])
df_wilaya = df_wilaya.drop([4, 6], axis=0) ## 
df_wilaya = df_wilaya.reset_index(drop=True)

## se añaden columnas y datos de forma manual de latitud, longitud y coordenadas
df_wilaya["latitud"] = ['27.73208098057244', '27.6146972796797', '27.50636207668466', '26.82570656731536', '27.49512461924248']
df_wilaya["longitud"] = ['-8.018310607785201', '-7.869766227805637', '-8.008280197426505', '-6.864677788854666', '-7.826538258457695']
df_wilaya["coordenadas"] = ['27.73208098057244, -8.018310607785201', '27.6146972796797, -7.869766227805637', '27.50636207668466, -8.008280197426505', '26.82570656731536, -6.864677788854666',
                            '27.49512461924248, -7.826538258457695']

## añadir nuevas wilayas
nueva_wilaya = pd.DataFrame({'wilaya': ['Rabuni'], 'latitud': ['27.4819532'], 'longitud': ['-8.0982475'], 'coordenadas': ['27.4819532, -8.0982475']}) ## tomar bdt o clm como coordenadas
df_wilaya = pd.concat([df_wilaya, nueva_wilaya], ignore_index=True)
no_wilaya = pd.DataFrame({'wilaya': ['No Wilaya']})
df_wilaya = pd.concat([df_wilaya, no_wilaya], ignore_index=True)

df_wilaya.reset_index(drop=True)
df_wilaya.index += 1
df_wilaya = df_wilaya.reset_index(names='id_wilaya')
df_wilaya

,id_wilaya,wilaya,latitud,longitud,coordenadas
0,1,Aaiun,27.73208098057244,-8.018310607785201,"27.73208098057244, -8.018310607785201"
1,2,Auserd,27.6146972796797,-7.869766227805637,"27.6146972796797, -7.869766227805637"
2,3,Bojador,27.50636207668466,-8.008280197426505,"27.50636207668466, -8.008280197426505"
3,4,Dajla,26.82570656731536,-6.864677788854666,"26.82570656731536, -6.864677788854666"
4,5,Smara,27.49512461924248,-7.826538258457695,"27.49512461924248, -7.826538258457695"
5,6,Rabuni,27.4819532,-8.0982475,"27.4819532, -8.0982475"
6,7,No Wilaya,NaN,NaN,NaN


### Maestro Tipo de Vehículo (tipo_vehiculo)

Este es un maestro que se genera de forma manual con los siguientes campos 

- id_tipo_vehiculo 
- tipo_vehiculo 
- descripcion

In [8]:
## Creación del DataFrame 

df_tipo_vehiculo = pd.DataFrame({'tipo_vehiculo': ['CC', 'CR', 'CA', 'CT', 'VA'], 'descripcion': ['Camion Cisterna', 'Camion Residuos', 'Camion Alimentos', 
                                 'Camion Transporte', '']})
df_tipo_vehiculo.index += 1
df_tipo_vehiculo = df_tipo_vehiculo.reset_index(names='id_tipo_vehiculo')
df_tipo_vehiculo

,id_tipo_vehiculo,tipo_vehiculo,descripcion
0,1,CC,Camion Cisterna
1,2,CR,Camion Residuos
2,3,CA,Camion Alimentos
3,4,CT,Camion Transporte
4,5,VA,


### Maestro Camión (camion)

Se tienen en consideración tres orígenes de maestros:

1. Información compartida por attsf en el drive `https://docs.google.com/spreadsheets/d/13H91tuPvpQV8UuIUpKO0sICA-zYiMDynrCn_qbxyneY/edit?usp=sharing` de camiones de institución **Acnur** Activos y Retirados.  
2. Maestro generado del ETL del notebook `srg_etl_notebook.ipynb` para el maestro camiones almacenado en Git de proyecto `app/data/processed/bbdd`.
3. Se toman datos de la información histótica desde el año 2019 de registros de distribuciones para  información optenida desde el sharepoint de ATTSF > 02. Datos > Bases de datos.


In [9]:
## maestro  (acnur)
df_camion_1_activo = pd.read_excel(
    path_input / 'Listado Flota y compras anuales.xlsx',
    sheet_name='Activos',
    usecols='A, B, C, E, D, F, G', 
    names=['tipo_vehiculo', 'marca', 'modelo', 'chasis', 'tag','nombre_attsf', 'wilaya'],
    header=14
)
df_camion_1_activo['estado'] = 'Activo'
df_camion_1_activo['institucion'] = 'Acnur'

## maestro  (acnur)
df_camion_1_retirado = pd.read_excel(
    path_input / 'Listado Flota y compras anuales.xlsx',
    sheet_name='Retirados',
    usecols='A, B, C, E, D, F, G, H', 
    names=['tipo_vehiculo','marca', 'modelo','chasis', 'tag', 'nombre_attsf', 'wilaya', 'fecha_retirada']
)
df_camion_1_retirado['estado'] = 'Retirado'
df_camion_1_retirado['institucion'] = 'Acnur'

df_camion_1 = pd.concat([df_camion_1_activo, df_camion_1_retirado])

## gestión de columnas de texto

df_camion_1['wilaya'] = df_camion_1['wilaya'].str.title()
df_camion_1['marca'] = df_camion_1['marca'].str.title()

## sustitución valores de wilaya 
df_camion_1 = df_camion_1.reset_index(drop=True)
## Bucle que unifica los datos en la columna wilaya 
for i in range(0, len(df_camion_1['wilaya'])):
    for key in d_unificados_wilaya.keys():
        for value in d_unificados_wilaya[key]:
            if value in df_camion_1.loc[i, 'wilaya']:
                df_camion_1.loc[i, 'wilaya'] = df_camion_1.loc[i, 'wilaya'].replace(value, key)

df_camion_1['nombre_attsf'] = df_camion_1['nombre_attsf'].astype(str)


In [10]:
df_camion_2 = pd.read_csv(path_output / 'bbdd/camion.csv')
df_camion_2 = df_camion_2.drop('Unnamed: 0', axis=1)

## se cambian las columnas de ingles a español 
columnas_camion_2 = ['nombre_attsf', 'wilaya', 'marca', 'estado', 'tag', 'registro', 'chasis', 'institucion', 'espec_motor', 'ano', 'capacidad']
df_camion_2.columns = columnas_camion_2
df_camion_2['tipo_vehiculo'] = ' '

## se eliminan valores duplicados 
df_camion_2 = df_camion_2.drop_duplicates()
df_camion_2 = df_camion_2.reset_index(drop=True)


## se asigna tipo_vehiculo
for i in range(0, len(df_camion_2['nombre_attsf'])): 
    if df_camion_2.loc[i,'nombre_attsf'].startswith('CC'):
        df_camion_2.loc[i, 'tipo_vehiculo'] = df_camion_2.loc[i, 'tipo_vehiculo'].replace(' ', 'CC')
    if df_camion_2.loc[i,'nombre_attsf'].startswith('CR'):
        df_camion_2.loc[i, 'tipo_vehiculo'] = df_camion_2.loc[i, 'tipo_vehiculo'].replace(' ', 'CR')
    if df_camion_2.loc[i,'nombre_attsf'].startswith(('TALLER', 'INST', 'AUT')): ## omitir 
        df_camion_2 = df_camion_2.drop(i, axis=0)
    else:
        df_camion_2.loc[i, 'tipo_vehiculo'] = df_camion_2.loc[i, 'tipo_vehiculo'].replace(' ', 'CT')

In [11]:
list_camion_1 = ['CT1', 'CT2', 'CT3', 'CT4', 'CT5', 'CT6', 'CT7']
list_camion_2 = ['CT01', 'CT02', 'CT03', 'CT04', 'CT05', 'CT06', 'CT07']

## unificar los valores de los camiones
for i in range(0 , len(list_camion_1)):
    indice_camion = df_camion_2[df_camion_2['nombre_attsf'] == list_camion_1[i]].index
    df_camion_2.loc[indice_camion, 'nombre_attsf'] = list_camion_2[i]


In [49]:
df_camion_2 = df_camion_2.drop(df_camion_2[(df_camion_2['nombre_attsf'] == 'CR225') &  (df_camion_2['wilaya'] == 'Auserd')].index, axis = 0)
df_camion_2 = df_camion_2.drop(df_camion_2[(df_camion_2['nombre_attsf'] == 'CR226') &  (df_camion_2['wilaya'] == 'Aaiun')].index, axis = 0)

In [12]:
df_camion_concat_1 = pd.merge(left = df_camion_2[['nombre_attsf', 'wilaya', 'marca', 'tag', 'tipo_vehiculo']], right = df_camion_1, how='outer', on='nombre_attsf')


def rellenar_x_y(df, col_x, col_y):
    for i in range(0, len(df[col_y])):    
     if pd.isna(df.loc[i, col_y]): 
         df.loc[i, col_y] = df.loc[i, col_x]

rellenar_x_y(df_camion_concat_1, 'tag_x', 'tag_y')
rellenar_x_y(df_camion_concat_1, 'marca_x', 'marca_y')
rellenar_x_y(df_camion_concat_1, 'wilaya_x', 'wilaya_y')
rellenar_x_y(df_camion_concat_1, 'tipo_vehiculo_x', 'tipo_vehiculo_y')

df_camion_concat_1

,nombre_attsf,wilaya_x,marca_x,tag_x,tipo_vehiculo_x,tipo_vehiculo_y,marca_y,modelo,chasis,tag_y,wilaya_y,estado,institucion,fecha_retirada
0,CC53,Aaiun,Volvo,559915.0,CC,CC,Volvo,FMX 400,YV2JSG0D1CA728439,559915.0,Aaiun,Retirado,Acnur,2023-03-15
1,CC54,Aaiun,Volvo,559916.0,CC,CC,Volvo,FMX 400,YV2JSG0D6CA728498,559916.0,Aaiun,Retirado,Acnur,2023-03-15
2,CC55,Aaiun,Volvo,559917.0,CC,CC,Volvo,NaN,NaN,559917.0,Aaiun,NaN,NaN,NaT
3,CC56,Aaiun,Volvo,559918.0,CC,CC,Volvo Plana,FMX 400,YV2JSG0D2CA729955,559918.0,Aaiun,Activo,Acnur,NaT
4,CC57,Aaiun,Volvo,559919.0,CC,CC,Volvo Plana,FMX 400,YV2JSG0DXCA730173,559919.0,Aaiun,Activo,Acnur,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,CR218,Aaiun,Man,771485.0,CR,CR,Man,TGM 18.240,WMAN08ZZ2MY420374,771485.0,Aaiun,Activo,Acnur,NaT
70,CR219,Smara,Man,771486.0,CR,CR,Man,TGM 18.240,WMAN08ZZ9MY420419,771486.0,Smara,Activo,Acnur,NaT
71,VA01,NaN,NaN,NaN,NaN,VA,Nissan,Patrol,JN1TESY61Z0576267,981367.0,Aux,Activo,Acnur,NaT
72,VA02,NaN,NaN,NaN,NaN,VA,Mercedes,UNIMOG,NaN,NaN,Aux,Activo,Acnur,NaT


In [13]:
## reconstrucción dataframe df_camion_concat_1 
df_camion_concat_1 = df_camion_concat_1[['nombre_attsf', 'tipo_vehiculo_y', 'marca_y', 'modelo', 'chasis', 'tag_y', 'wilaya_y', 'estado', 'institucion', 'fecha_retirada']]
df_camion_concat_1.columns = ['nombre_attsf', 'tipo_vehiculo', 'marca', 'modelo', 'chasis', 'tag', 'wilaya', 'estado', 'institucion', 'fecha_retirada']
df_camion_concat_1


,nombre_attsf,tipo_vehiculo,marca,modelo,chasis,tag,wilaya,estado,institucion,fecha_retirada
0,CC53,CC,Volvo,FMX 400,YV2JSG0D1CA728439,559915.0,Aaiun,Retirado,Acnur,2023-03-15
1,CC54,CC,Volvo,FMX 400,YV2JSG0D6CA728498,559916.0,Aaiun,Retirado,Acnur,2023-03-15
2,CC55,CC,Volvo,NaN,NaN,559917.0,Aaiun,NaN,NaN,NaT
3,CC56,CC,Volvo Plana,FMX 400,YV2JSG0D2CA729955,559918.0,Aaiun,Activo,Acnur,NaT
4,CC57,CC,Volvo Plana,FMX 400,YV2JSG0DXCA730173,559919.0,Aaiun,Activo,Acnur,NaT
...,...,...,...,...,...,...,...,...,...,...
69,CR218,CR,Man,TGM 18.240,WMAN08ZZ2MY420374,771485.0,Aaiun,Activo,Acnur,NaT
70,CR219,CR,Man,TGM 18.240,WMAN08ZZ9MY420419,771486.0,Smara,Activo,Acnur,NaT
71,VA01,VA,Nissan,Patrol,JN1TESY61Z0576267,981367.0,Aux,Activo,Acnur,NaT
72,VA02,VA,Mercedes,UNIMOG,NaN,NaN,Aux,Activo,Acnur,NaT


In [14]:
## maestro 3 
## lista para almacenar los datos de cada archivo de Excel
data_list = []

## bucle para leer cada archivo de Excel en la carpeta y almacenar los datos en data_list
for file_name in os.listdir(path_input_historic):
    if file_name.endswith('.xlsx'): # comprobar si el archivo es un archivo de Excel
        data = pd.read_excel(
            path_input_historic / file_name,
            sheet_name = 'base  datos', ## optimizar búsqueda de hoja
            usecols='C',
            names='nombre_attsf', 
            header=3
        )
        data_list.append(data)

## combinar todos los datos en un solo DataFrame
df_camion_3 = pd.concat(data_list, axis=0, ignore_index=True).dropna(how='all')
df_camion_3.columns = ['nombre_attsf']

## se detecta que hay valores con columnas en ','
df_camion_3 = df_camion_3.drop(df_camion_3[df_camion_3['nombre_attsf'] == ","].index, axis=0)
## se detecta que hay una fila con datos vacios en la columna camion, la cual se elimina 
df_camion_3 = df_camion_3.drop(df_camion_3[df_camion_3['nombre_attsf'] == "          "].index, axis=0)

## se almacenan valores unicos 
df_camion_3 = pd.DataFrame(df_camion_3['nombre_attsf'].unique(), columns=['nombre_attsf']).sort_values('nombre_attsf') 
df_camion_3['nombre_attsf'] = df_camion_3['nombre_attsf'].astype(int)
df_camion_3['tipo_vehiculo'] = np.repeat('CA', len(df_camion_3['nombre_attsf']), axis=0)
df_camion_3['wilaya'] = np.repeat('No Wilaya', len(df_camion_3['nombre_attsf']), axis=0)

In [15]:
## se juntan todos los maestros 
df_camion_concat_2 = pd.concat([df_camion_concat_1, df_camion_3])
df_camion_concat_2['nombre_attsf'] = df_camion_concat_2['nombre_attsf'].astype(str)
#df_camion_concat = df_camion_concat.sort_values('nombre_attsf')
df_camion_concat_2 = df_camion_concat_2.reset_index(drop=True)
df_camion_concat_2

,nombre_attsf,tipo_vehiculo,marca,modelo,chasis,tag,wilaya,estado,institucion,fecha_retirada
0,CC53,CC,Volvo,FMX 400,YV2JSG0D1CA728439,559915.0,Aaiun,Retirado,Acnur,2023-03-15
1,CC54,CC,Volvo,FMX 400,YV2JSG0D6CA728498,559916.0,Aaiun,Retirado,Acnur,2023-03-15
2,CC55,CC,Volvo,NaN,NaN,559917.0,Aaiun,NaN,NaN,NaT
3,CC56,CC,Volvo Plana,FMX 400,YV2JSG0D2CA729955,559918.0,Aaiun,Activo,Acnur,NaT
4,CC57,CC,Volvo Plana,FMX 400,YV2JSG0DXCA730173,559919.0,Aaiun,Activo,Acnur,NaT
...,...,...,...,...,...,...,...,...,...,...
87,23,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT
88,24,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT
89,25,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT
90,26,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT


In [16]:
## se modifican los nombres de los vehiculo de tipo_vehiculo == CA
indice_ca = df_camion_concat_2[df_camion_concat_2['tipo_vehiculo'] == 'CA'].index
df_camion_concat_2.loc[indice_ca, 'nombre_attsf'] = df_camion_concat_2.loc[indice_ca, 'tipo_vehiculo'] + df_camion_concat_2.loc[indice_ca, 'nombre_attsf']


In [17]:
df_camion_concat_2['propietario'] = ''
indice_propietario_1 = df_camion_concat_2[(df_camion_concat_2['wilaya'] == 'Inst') | (df_camion_concat_2['wilaya'] == 'Instituciones') | (df_camion_concat_2['wilaya'] == 'Transport') |
                                        (df_camion_concat_2['wilaya'] == 'Aux')].index
df_camion_concat_2.loc[indice_propietario_1, 'propietario'] = df_camion_concat_2.loc[indice_propietario_1, 'wilaya']
df_camion_concat_2.loc[indice_propietario_1, 'wilaya'] = df_wilaya.loc[5, 'wilaya']

In [56]:
indice_propietario_2 = indice_propietario_2 = df_camion_concat_2[df_camion_concat_2['propietario'] == ''].index
df_camion_concat_2.loc[indice_propietario_2, 'propietario'] = 'Attsf'

In [18]:
df_camion_concat_2[(df_camion_concat_2['wilaya'] == 'Inst') | (df_camion_concat_2['wilaya'] == 'Instituciones') | (df_camion_concat_2['wilaya'] == 'Transport') |
                                        (df_camion_concat_2['wilaya'] == 'Aux')]

,nombre_attsf,tipo_vehiculo,marca,modelo,chasis,tag,wilaya,estado,institucion,fecha_retirada,propietario


In [19]:
df_camion_concat_2[(df_camion_concat_2['propietario'] == 'Inst') | (df_camion_concat_2['propietario'] == 'Instituciones') | (df_camion_concat_2['propietario'] == 'Transport') |
                                        (df_camion_concat_2['propietario'] == 'Aux')]

,nombre_attsf,tipo_vehiculo,marca,modelo,chasis,tag,wilaya,estado,institucion,fecha_retirada,propietario
7,CC74,CC,Iveco,NaN,VF622AXA000103293,NaN,Rabuni,Activo,Acnur,NaT,Inst
31,CC02,CC,Sinotruck,HOWO335,LZZ5BLNF7KN507613,559994.0,Rabuni,Activo,Acnur,NaT,Inst
38,CR140,CR,Man,TGL 10-180,WMAN05ZZ9HY362333,614224.0,Rabuni,Activo,Acnur,NaT,Inst
45,CC79,CC,Renault,NaN,NaN,NaN,Rabuni,NaN,NaN,NaT,Instituciones
46,CC81,CC,Iveco,NaN,NaN,NaN,Rabuni,Activo,Acnur,NaT,Inst
47,CC82,CC,Volvo Plana,FL7,YB165A3A7PB102289,NaN,Rabuni,Retirado,Acnur,2023-03-15,Inst
48,CC09,CC,Mercedes,AXOR 1840,WDB9440331K756391,NaN,Rabuni,Retirado,Acnur,2023-03-15,Inst
55,CT01,CT,Volvo Plana,NL10,YW2D2B3D3WA277966,NaN,Rabuni,Activo,Acnur,NaT,Transport
56,CT02,CT,Volvo Plana,NL10,YW2D2B3D5WA2277967,NaN,Rabuni,Activo,Acnur,NaT,Transport
57,CT03,CT,Renault,260,VF633BXB000100296,NaN,Rabuni,Activo,Acnur,NaT,Transport


In [20]:
df_camion_concat_2['wilaya'].unique()

array(['Aaiun', 'Rabuni', 'Dajla', 'Auserd', 'Bojador', 'Smara',
       'No Wilaya'], dtype=object)

In [21]:
## merge de la tabla camion con la tabla wilaya
df_camion_merge = pd.merge(left = df_camion_concat_2, right = df_wilaya.loc[:, ['id_wilaya', 'wilaya']], how='left', on='wilaya')

## merge de la tabla camion con la tabla tipo_vehiculo
df_camion_merge = pd.merge(left = df_camion_merge, right = df_tipo_vehiculo[['id_tipo_vehiculo', 'tipo_vehiculo']], how='left', on='tipo_vehiculo')

df_camion_merge

,nombre_attsf,tipo_vehiculo,marca,modelo,chasis,tag,wilaya,estado,institucion,fecha_retirada,propietario,id_wilaya,id_tipo_vehiculo
0,CC53,CC,Volvo,FMX 400,YV2JSG0D1CA728439,559915.0,Aaiun,Retirado,Acnur,2023-03-15,,1,1
1,CC54,CC,Volvo,FMX 400,YV2JSG0D6CA728498,559916.0,Aaiun,Retirado,Acnur,2023-03-15,,1,1
2,CC55,CC,Volvo,NaN,NaN,559917.0,Aaiun,NaN,NaN,NaT,,1,1
3,CC56,CC,Volvo Plana,FMX 400,YV2JSG0D2CA729955,559918.0,Aaiun,Activo,Acnur,NaT,,1,1
4,CC57,CC,Volvo Plana,FMX 400,YV2JSG0DXCA730173,559919.0,Aaiun,Activo,Acnur,NaT,,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,CA23,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT,,7,3
88,CA24,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT,,7,3
89,CA25,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT,,7,3
90,CA26,CA,NaN,NaN,NaN,NaN,No Wilaya,NaN,NaN,NaT,,7,3


In [22]:
## Reconstrucción de dataframe 
df_camion = pd.DataFrame({'nombre_attsf': df_camion_merge['nombre_attsf'], 'id_wilaya': df_camion_merge['id_wilaya'], 'id_tipo_vehiculo': df_camion_merge['id_tipo_vehiculo'],
                          'marca':df_camion_merge['marca'], 'modelo': df_camion_merge['modelo'], 'estado': df_camion_merge['estado'], 'numero_etiqueta': df_camion_merge['tag'], 
                          'registro': '', 'chasis': df_camion_merge['chasis'], 'institucion': df_camion_merge['institucion'], 'espec_motor': '',
                          'ano': '', 'capacidad': '', 'fecha_alta': '', 'fecha_baja': df_camion_merge['fecha_retirada'], 'propietario': df_camion_merge['propietario']
                        }) 
df_camion

,nombre_attsf,id_wilaya,id_tipo_vehiculo,marca,modelo,estado,numero_etiqueta,registro,chasis,institucion,espec_motor,ano,capacidad,fecha_alta,fecha_baja,propietario
0,CC53,1,1,Volvo,FMX 400,Retirado,559915.0,,YV2JSG0D1CA728439,Acnur,,,,,2023-03-15,
1,CC54,1,1,Volvo,FMX 400,Retirado,559916.0,,YV2JSG0D6CA728498,Acnur,,,,,2023-03-15,
2,CC55,1,1,Volvo,NaN,NaN,559917.0,,NaN,NaN,,,,,NaT,
3,CC56,1,1,Volvo Plana,FMX 400,Activo,559918.0,,YV2JSG0D2CA729955,Acnur,,,,,NaT,
4,CC57,1,1,Volvo Plana,FMX 400,Activo,559919.0,,YV2JSG0DXCA730173,Acnur,,,,,NaT,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,CA23,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
88,CA24,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
89,CA25,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
90,CA26,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,


In [23]:

 ##########################################################################################################
df_camion.reset_index(drop='True')
df_camion.index += 1
df_camion = df_camion.reset_index(names='id_camion')
df_camion

,id_camion,nombre_attsf,id_wilaya,id_tipo_vehiculo,marca,modelo,estado,numero_etiqueta,registro,chasis,institucion,espec_motor,ano,capacidad,fecha_alta,fecha_baja,propietario
0,1,CC53,1,1,Volvo,FMX 400,Retirado,559915.0,,YV2JSG0D1CA728439,Acnur,,,,,2023-03-15,
1,2,CC54,1,1,Volvo,FMX 400,Retirado,559916.0,,YV2JSG0D6CA728498,Acnur,,,,,2023-03-15,
2,3,CC55,1,1,Volvo,NaN,NaN,559917.0,,NaN,NaN,,,,,NaT,
3,4,CC56,1,1,Volvo Plana,FMX 400,Activo,559918.0,,YV2JSG0D2CA729955,Acnur,,,,,NaT,
4,5,CC57,1,1,Volvo Plana,FMX 400,Activo,559919.0,,YV2JSG0DXCA730173,Acnur,,,,,NaT,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,CA23,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
88,89,CA24,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
89,90,CA25,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,
90,91,CA26,7,3,NaN,NaN,NaN,NaN,,NaN,NaN,,,,,NaT,


## Unificación Tabla Hechos Distribución 

In [93]:
## Path de archivos históricos 
path_input_historic = dependencies.rootFolder / 'data' / 'raw_historic'
path_output_historic = dependencies.rootFolder / 'data' / 'processed_historic'

In [191]:
# lista para almacenar los datos de cada archivo de Excel
data_list = []

# bucle para leer cada archivo de Excel en la carpeta y almacenar los datos en data_list
for file_name in os.listdir(path_input_historic):
    if file_name.endswith('.xlsx'): # comprobar si el archivo es un archivo de Excel
        # file_path = os.path.join(path, file_name)
        data = pd.read_excel(
            path_input_historic / file_name,
            sheet_name = 'base  datos',
            usecols='A, B, C, D, E, F, G, I, J, K, L, M, N, AC, AD',
            names=['no_serie', 'conductor', 'nombre_attsf','fecha_salida', 'hora_salida', 'fecha_llegada', 'hora_llegada', 'km_salida', 'km_llegada', 'km_totales',
                'tm', 'tipo_producto', 'wilaya', 'incidencias', 'observaciones'], 
            header=2
        )
        data_list.append(data)

# combinar todos los datos en un solo DataFrame
df_distribucion_concat = pd.concat(data_list, axis=0, ignore_index=True)
## Eliminar filas con todas las columnas nulas de 10296 se reducen a 5691 filas 
df_distribucion_concat = df_distribucion_concat.dropna(how='all')
## Se detecta que hay valores con "," en la columna camion, lo cuales se eliminan 
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'] == ","].index, axis=0)
## Se detecta que hay una fila con datos vacios en la columna camion, la cual se elimina 
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'] == "          "].index, axis=0)

## Las columnas que pueden admitir valores nulos son [km_salida, km_llegada, incidencias, observaciones]
## Se verifican si las columnas que no admiten valores nulos los tienen 
## Se verifican valores nulos 
#print(df_distribucion['no_serie'].isnull().sum())
#print(df_distribucion['id_conductor'].isnull().sum())
#print(df_distribucion['id_camion'].isnull().sum())
#print(df_distribucion['salida_fecha_hora'].isnull().sum())
#print(df_distribucion['llegada_fecha_hora'].isnull().sum())
#print(df_distribucion['tm'].isnull().sum())
#print(df_distribucion['id_tipo_producto'].isnull().sum())
#print(df_distribucion['id_wilaya'].isnull().sum())

## podriamos eliminar las filas de los índices en los cuales hay valores nulos igual a 37
## pero se ha visto que esto correpsonde a un conductor el cual si tiene registros de distribución, 
## por lo pronto se borran solo los indices contenitos en las filas donde para camion el valor es null 
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'].isnull()].index, axis=0)
## quedando solamente aquella columna con el registro del condutor perdido 
## Esto hace que las columnas que no pueden tener valores nuelos queden en 0 excepto aquella con el conductor perdido 

## Se resetea el índice 
df_distribucion_concat = df_distribucion_concat.reset_index(drop=True)

# ## las columnas de fecha_salida y fecha_llegada deben de tener formato 'datetime.time' sin embargo se detecta que hay unas con formato 'datetime.datetime' 
# ## Se obtendrán aquellas que no corresponden con el formato
import datetime

df_distribucion_concat.loc[0, 'hora_llegada'] = datetime.time(12, 0)
#df_distribucion_concat.loc[0, 'hora_llegada']

## Se ha hecho una modificación manual en el primer registro en la hora de llegada en la hoja excel por inconcistencia en el tipo de dato 
df_distribucion_concat['salida_fecha_hora'] = pd.to_datetime(df_distribucion_concat['fecha_salida']).dt.date.astype(str) + ' ' + df_distribucion_concat['hora_salida'].astype(str)
df_distribucion_concat['llegada_fecha_hora'] = pd.to_datetime(df_distribucion_concat['fecha_llegada']).dt.date.astype(str) + ' ' + df_distribucion_concat['hora_llegada'].astype(str)

## Gestion de columnas de texto 
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.title()
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.lstrip()
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.rstrip()

df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.title()
df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.lstrip()
df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.rstrip()

## Bucle que unifica los datos en la columna wilaya 
for i in range(0, len(df_distribucion_concat['wilaya'])):
     for key in d_unificados_wilaya.keys():
         for value in d_unificados_wilaya[key]:
             if value in df_distribucion_concat.loc[i, 'wilaya']:
                 df_distribucion_concat.loc[i, 'wilaya'] = df_distribucion_concat.loc[i, 'wilaya'].replace(value, key)

## Se verifica que los valores que hay en la columna wilaya de df_distribucion_concat correspondan con el maestro wilaya 
# df_distribucion_concat['wilaya'].unique()   

## se detecta fila con el conductor perdido 
df_conductor_miss = df_distribucion_concat[df_distribucion_concat['conductor'].isnull()]
ind_conductor_miss = df_distribucion_concat[df_distribucion_concat['conductor'].isnull()].index

## columna con el valor del conductor nulo que se convierte a str para que pueda iterar 
#df_distribucion_concat.loc[ind_conductor_miss, 'conductor'] = str(df_distribucion_concat.loc[ind_conductor_miss, 'conductor'])
df_distribucion_concat = df_distribucion_concat.drop(ind_conductor_miss, axis=0)
df_distribucion_concat = df_distribucion_concat.reset_index(drop=True)

## se detecta que hay valores en los nombres de los conductores inconsistentes, de acuerdo con los criterior de desarrollo de maestro conductor se unifican los datos 
#print(df_distribucion_concat['conductor'].unique())
#print(len(df_distribucion_concat['conductor'].unique()))

## Bucle que unifica los datos de la columna conductor en el df_distribucion_concat 
for i in range(0, len(df_distribucion_concat['conductor'])):
     for key in d_unificados_conductor.keys():
         for value in d_unificados_conductor[key]:
             if value in df_distribucion_concat.loc[i, 'conductor']:
                 df_distribucion_concat.loc[i, 'conductor'] = df_distribucion_concat.loc[i, 'conductor'].replace(value, key)

df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(int)

## bucle que unifica nombres de camiones de tipo_vehiculo == CA 
for i in range (0, len(df_distribucion_concat['nombre_attsf'])):
    df_distribucion_concat.loc[i, 'nombre_attsf'] = 'CA' + str(df_distribucion_concat.loc[i, 'nombre_attsf'])

df_distribucion_concat

,no_serie,conductor,nombre_attsf,fecha_salida,hora_salida,fecha_llegada,hora_llegada,km_salida,km_llegada,km_totales,tm,tipo_producto,wilaya,incidencias,observaciones,salida_fecha_hora,llegada_fecha_hora
0,1.0,Seilum,CA10,2023-01-02,09:00:00,2023-01-02,12:00:00,NaN,NaN,335.0,12.0,CB,Dajla,NaN,NaN,2023-01-02 09:00:00,2023-01-02 12:00:00
1,2.0,Bolla,CA11,2023-01-02,09:00:00,2023-01-02,17:13:00,NaN,NaN,335.0,12.0,CB,Dajla,NaN,NaN,2023-01-02 09:00:00,2023-01-02 17:13:00
2,3.0,Ahmed Salem,CA12,2023-01-02,09:00:00,2023-01-02,18:21:00,NaN,NaN,335.0,8.0,CB,Dajla,NaN,NaN,2023-01-02 09:00:00,2023-01-02 18:21:00
3,4.0,Embarec Ahmed,CA13,2023-01-02,08:40:00,2023-01-02,16:08:00,NaN,NaN,335.0,10.0,CB,Dajla,NaN,NaN,2023-01-02 08:40:00,2023-01-02 16:08:00
4,5.0,Ramdan,CA18,2023-01-02,09:00:00,2023-01-02,16:00:00,NaN,NaN,106.0,8.0,CB,Aaiun,NaN,NaN,2023-01-02 09:00:00,2023-01-02 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5640,567.0,Embarec Ahmed,CA21,2021-03-23,09:10:00,2021-03-23,13:09:00,192295.0,192368,73.0,12.5,CB,Auserd,NaN,NaN,2021-03-23 09:10:00,2021-03-23 13:09:00
5641,568.0,Lajlifa Moh. Fadel,CA22,2021-03-23,09:00:00,2021-03-23,17:55:00,172506.0,172587,81.0,22.5,CB,Auserd,NaN,NaN,2021-03-23 09:00:00,2021-03-23 17:55:00
5642,569.0,Mohamed Baba,CA26,2021-03-23,09:00:00,2021-03-23,16:00:00,194000.0,194082,82.0,25.0,CB,Auserd,NaN,NaN,2021-03-23 09:00:00,2021-03-23 16:00:00
5643,570.0,Chej Hamdeiti,CA13,2021-03-23,09:00:00,2021-03-23,19:06:00,NaN,NaN,106.0,10.0,CB,Aaiun,NaN,NaN,2021-03-23 09:00:00,2021-03-23 19:06:00


In [192]:
# ## Se verifican los valores quedando unicamente 26 valores únicos, ## Se tiene que agregar un nuevo conductor al maestro de conductores, 
# ## se omite porque se ha extendido la muestra del histórico de maestros
# print(df_distribucion_concat['conductor'].unique())
# print(len(df_distribucion_concat['conductor'].unique()))   
             

## merge de la tabla distribuciones con la tabla tipo_producto 
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_tipo_producto, how='outer', left_on='tipo_producto', 
right_on='tipo_producto')

## merge de la tabla distribuciones con la tabla conductor
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_conductor, how='outer')

#df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(int)
df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(str)

## merge de la tabla distribuciones con la tabla camion
## hay problema con el tipo de datos de columna camion de df_distribucion_concat 
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_camion[['id_camion', 'nombre_attsf']], how='left', left_on='nombre_attsf', right_on='nombre_attsf')

## merge de la tabla distribuciones con la tabla wilaya
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_wilaya[['id_wilaya', 'wilaya']], how='inner')

# ## se detecta que no hay valores nulos, excepto uno para la columna conductor 
# df_distribucion_concat['id_tipo_producto'].isnull().sum()
# df_distribucion_concat['id_conductor'].isnull().sum()
# df_distribucion_concat['id_camion'].isnull().sum()
# df_distribucion_concat['id_wilaya'].isnull().sum()

df_distribucion_concat

,no_serie,conductor,nombre_attsf,fecha_salida,hora_salida,fecha_llegada,hora_llegada,km_salida,km_llegada,km_totales,...,wilaya,incidencias,observaciones,salida_fecha_hora,llegada_fecha_hora,id_tipo_producto,descripcion,id_conductor,id_camion,id_wilaya
0,1.0,Seilum,CA10,2023-01-02,09:00:00,2023-01-02,12:00:00,NaN,NaN,335.0,...,Dajla,NaN,NaN,2023-01-02 09:00:00,2023-01-02 12:00:00,1,Canasta Basica,23,73,4
1,149.0,Seilum,CA10,2023-01-20,09:00:00,2023-01-20,18:30:00,NaN,NaN,335.0,...,Dajla,NaN,NaN,2023-01-20 09:00:00,2023-01-20 18:30:00,1,Canasta Basica,23,73,4
2,46.0,Seilum,CA10,2019-01-08,09:00:00,2019-01-08,18:30:00,NaN,NaN,335.0,...,Dajla,NaN,NaN,2019-01-08 09:00:00,2019-01-08 18:30:00,1,Canasta Basica,23,73,4
3,206.0,Seilum,CA10,2019-02-06,09:00:00,2019-02-06,16:40:00,NaN,NaN,335.0,...,Dajla,NaN,NaN,2019-02-06 09:00:00,2019-02-06 16:40:00,1,Canasta Basica,23,73,4
4,423.0,Seilum,CA10,2019-03-06,09:00:00,2019-03-06,19:00:00,NaN,NaN,335.0,...,Dajla,NaN,NaN,2019-03-06 09:00:00,2019-03-06 19:00:00,1,Canasta Basica,23,73,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5640,7.0,Iarbana Hamudi,CA19,2021-01-02,09:00:00,2021-01-02,13:06:00,NaN,NaN,30.0,...,Bojador,NaN,NaN,2021-01-02 09:00:00,2021-01-02 13:06:00,1,Canasta Basica,10,82,3
5641,386.0,Iarbana Hamudi,CA22,2021-02-21,09:00:00,2021-02-21,13:00:00,NaN,NaN,8.0,...,Bojador,NaN,NaN,2021-02-21 09:00:00,2021-02-21 13:00:00,1,Canasta Basica,10,85,3
5642,506.0,Iarbana Hamudi,CA17,2021-03-15,09:00:00,2021-03-15,13:10:00,NaN,NaN,30.0,...,Bojador,NaN,NaN,2021-03-15 09:00:00,2021-03-15 13:10:00,1,Canasta Basica,10,80,3
5643,1432.0,Iarbana Hamudi,CA20,2020-07-25,04:00:00,2020-07-25,08:45:00,220158,220191.0,33.0,...,Bojador,NaN,NaN,2020-07-25 04:00:00,2020-07-25 08:45:00,2,Producto Fresco,10,83,3


In [193]:
## reconstrucción de información histórica distribución

# se arma de nuevo el dataframe distribuciones 
d_distribucion = {'id_conductor': df_distribucion_concat['id_conductor'], 'id_tipo_producto': df_distribucion_concat['id_tipo_producto'], 'id_camion': df_distribucion_concat['id_camion'],  
    'id_wilaya': df_distribucion_concat['id_wilaya'], 'no_serie': df_distribucion_concat['no_serie'],
    'salida_fecha_hora': df_distribucion_concat['salida_fecha_hora'], 'llegada_fecha_hora': df_distribucion_concat['llegada_fecha_hora'], 
    'km_salida': df_distribucion_concat['km_salida'], 'km_llegada': df_distribucion_concat['km_llegada'], 'km_totales': df_distribucion_concat['km_totales'], 'tm': df_distribucion_concat['tm'], 
    'incidencias': df_distribucion_concat['incidencias'], 'observaciones': df_distribucion_concat['observaciones']}
df_distribucion= pd.DataFrame(d_distribucion) 


## ordenar valores por fecha 
df_distribucion = df_distribucion.sort_values('salida_fecha_hora')


## resetear index 
df_distribucion = df_distribucion.reset_index(drop=True)
df_distribucion.index +=1 
df_distribucion = df_distribucion.reset_index(names='id_distribucion')

df_distribucion['no_serie'] = df_distribucion['no_serie'].astype(int)
df_distribucion['id_tipo_producto'] = df_distribucion['id_tipo_producto'].astype(int)
df_distribucion['id_wilaya'] = df_distribucion['id_wilaya'].astype(int)
#df_distribucion['id_camion'] = df_distribucion['id_camion'].astype(int)
df_distribucion['id_conductor'] = df_distribucion['id_conductor'].astype(int) 



In [203]:
#df_distribucion[type(df_distribucion['km_salida']) == float]


,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
2474,2475,10,1,83,1,9,2020-01-04 09:00:00,2020-01-04 16:15:00,209929,210024.0,95.0,11.0,NaN,NaN
2479,2480,16,1,89,1,12,2020-01-04 09:00:00,2020-01-04 17:00:00,169556,169656.0,100.0,11.0,NaN,NaN
2482,2483,8,1,84,1,10,2020-01-04 09:14:00,2020-01-04 18:38:00,71189,71306.0,117.0,20.0,NaN,NaN
2483,2484,12,1,85,1,11,2020-01-04 09:30:00,2020-01-04 18:25:00,147253,147375.0,122.0,20.0,NaN,NaN
2486,2487,1,1,87,1,25,2020-01-05 08:53:00,2020-01-05 16:00:00,149495,149603.0,108.0,11.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861,4862,1,1,87,5,2393,2020-12-30 09:00:00,2020-12-30 17:30:00,170041,170113.0,72.0,20.0,NaN,NaN
4863,4864,13,1,83,3,2401,2020-12-31 08:00:00,2020-12-31 17:17:00,227382,227410.0,28.0,12.0,NaN,NaN
4864,4865,12,1,85,5,2403,2020-12-31 08:30:00,2020-12-31 16:20:00,166540,166615.0,75.0,20.0,NaN,NaN
4866,4867,16,1,89,4,2405,2020-12-31 09:00:00,2020-12-31 18:30:00,188108,188424.0,316.0,20.0,NaN,NaN


In [186]:
df_distribucion.loc[2818, 'llegada_fecha_hora'] = '2020-02-22 03:00:00'
df_distribucion.loc[5458, 'llegada_fecha_hora'] = '2023-01-04 12:00:00' 
df_distribucion.loc[658, 'salida_fecha_hora'] = '2019-04-04 08:00:00'
df_distribucion.loc[740, 'tm'] = 4.0
df_distribucion.loc[df_distribucion[df_distribucion['km_salida'] == ' '].index, 'km_salida'] = float('nan')
df_distribucion.loc[3884, 'km_salida'] = float('nan') ## tenian espacios super largos
df_distribucion.loc[5086,'km_llegada'] = float('nan') ## tenian espacios super largos 

df_distribucion

In [160]:
## Se verifican valores nulos 
print(df_distribucion['no_serie'].isnull().sum())
print(df_distribucion['id_conductor'].isnull().sum())
print(df_distribucion['id_camion'].isnull().sum())
print(df_distribucion['salida_fecha_hora'].isnull().sum())
print(df_distribucion['llegada_fecha_hora'].isnull().sum())
print(df_distribucion['tm'].isnull().sum())
print(df_distribucion['id_tipo_producto'].isnull().sum())
print(df_distribucion['id_wilaya'].isnull().sum())

0
0
0
0
0
0
0
0


In [188]:
## Guardar tabla hechos en ficheros .CSV
# df_tipo_producto.to_csv(path_output / 'tipo_producto.csv', index=False, encoding = 'latin1')
# df_conductor.to_csv(path_output / 'conductor.csv', index=False, encoding='latin1')
# df_wilaya.to_csv(path_output / 'wilaya.csv', index=False, encoding = 'latin1')
# df_tipo_vehiculo.to_csv(path_output / 'tipo_vehiculo.csv', index=False, encoding = 'latin1')
# df_camion.to_csv(path_output / 'camion.csv', index=False, encoding='latin1')
# df_distribucion.to_csv(path_output / 'distribucion.csv', index=False, encoding = 'latin1')

## Agregar datos de distribución del 2022 

In [24]:
## Path de archivos históricos 
path_input_historic_copia = dependencies.rootFolder / 'data' / 'raw_historic - copia'

In [25]:
# lista para almacenar los datos de cada archivo de Excel
lista_anos = ['2019', '2020', '2021', '2022', '2023']
data_list = []

# bucle para leer cada archivo de Excel en la carpeta y almacenar los datos en data_list
for i, file_name in zip(lista_anos, os.listdir(path_input_historic_copia)):
    if file_name.endswith('.xlsx'): # comprobar si el archivo es un archivo de Excel
        file_path = os.path.join(path_input_historic_copia, file_name)
        data = pd.read_excel(
            path_input_historic_copia / file_name,
            sheet_name = 'base  datos ' + i,
            usecols='A, B, C, D, E, F, G, I, J, K, L, M, N, AC, AD',
            names=['no_serie', 'conductor', 'nombre_attsf','fecha_salida', 'hora_salida', 'fecha_llegada', 'hora_llegada', 'km_salida', 'km_llegada', 'km_totales',
                'tm', 'tipo_producto', 'wilaya', 'incidencias', 'observaciones'], 
            header=2
        )
        data_list.append(data)

# combinar todos los datos en un solo DataFrame
df_distribucion_concat = pd.concat(data_list, axis=0, ignore_index=True)

In [26]:
## Eliminar filas con todas las columnas nulas de 10296 se reducen a 5691 filas 
df_distribucion_concat = df_distribucion_concat.dropna(how='all')
## Se detecta que hay valores con "," en la columna camion, lo cuales se eliminan 
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'] == ","].index, axis=0)
## Se detecta que hay una fila con datos vacios en la columna camion, la cual se elimina 
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'] == "          "].index, axis=0)


In [27]:
df_distribucion_concat = df_distribucion_concat.drop(df_distribucion_concat[df_distribucion_concat['nombre_attsf'].isnull()].index, axis=0)

In [28]:
df_distribucion_concat

,no_serie,conductor,nombre_attsf,fecha_salida,hora_salida,fecha_llegada,hora_llegada,km_salida,km_llegada,km_totales,tm,tipo_producto,wilaya,incidencias,observaciones
0,1,seilum,10,2019-01-02,09:00:00,2019-01-02,15:42:00,NaN,NaN,106.0,11.85,CB,Aaiun,NaN,NaN
1,2,Said Al-lal,14,2019-01-02,09:00:00,2019-01-02,16:38:00,NaN,NaN,106.0,6,CB,Aaiun,NaN,NaN
2,3,Said Nafi,17,2019-01-02,09:00:00,2019-01-02,18:10:00,NaN,NaN,106.0,5,CB,Aaiun,NaN,NaN
3,4,Limam Moh. Mehdi,20,2019-01-02,09:00:00,2019-01-02,17:17:00,NaN,NaN,106.0,12,CB,Aaiun,NaN,NaN
4,5,Embarec Ahmed,21,2019-01-02,09:00:00,2019-01-02,16:37:00,52957.0,53065.0,108.0,20,CB,Aaiun,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10358,195.0,Abbelmula,24.0,2023-01-30,08:00:00,2023-01-30,16:39:00,199812.0,200141.0,329.0,20.0,E,Dajla,NaN,NaN
10359,196.0,Salma Mujtar,25.0,2023-01-30,08:00:00,2023-01-30,16:30:00,135152.0,135184.0,32.0,20.0,E,Bujador,NaN,NaN
10360,197.0,Hasana Lehbib,26.0,2023-01-30,08:00:00,2023-01-30,17:00:00,228199.0,228271.0,72.0,20.0,E,Smara,NaN,NaN
10361,198.0,Bolla,11.0,2023-01-31,08:45:00,2023-01-31,16:00:00,NaN,NaN,77.0,10.0,CB,Smara,NaN,NaN


In [29]:
## comoprobar tipo de datos de fecha 
df_distribucion_concat.dtypes


no_serie                 object
conductor                object
nombre_attsf             object
fecha_salida     datetime64[ns]
hora_salida              object
fecha_llegada    datetime64[ns]
hora_llegada             object
km_salida                object
km_llegada               object
km_totales               object
tm                       object
tipo_producto            object
wilaya                   object
incidencias              object
observaciones           float64
dtype: object

In [519]:
## Se verifican valores nulos 
print(df_distribucion_concat['no_serie'].isnull().sum())
print(df_distribucion_concat['conductor'].isnull().sum())
print(df_distribucion_concat['nombre_attsf'].isnull().sum())
print(df_distribucion_concat['tm'].isnull().sum())
print(df_distribucion_concat['tipo_producto'].isnull().sum())
print(df_distribucion_concat['wilaya'].isnull().sum())


0
1
0
0
0
0


In [30]:
## se detecta fila con el conductor perdido 
df_conductor_miss = df_distribucion_concat[df_distribucion_concat['conductor'].isnull()]
ind_conductor_miss = df_distribucion_concat[df_distribucion_concat['conductor'].isnull()].index
df_distribucion_concat = df_distribucion_concat.drop(ind_conductor_miss, axis=0)

In [521]:
print(df_distribucion_concat['no_serie'].isnull().sum())
print(df_distribucion_concat['conductor'].isnull().sum())
print(df_distribucion_concat['nombre_attsf'].isnull().sum())
print(df_distribucion_concat['tm'].isnull().sum())
print(df_distribucion_concat['tipo_producto'].isnull().sum())
print(df_distribucion_concat['wilaya'].isnull().sum())

0
0
0
0
0
0


In [31]:
df_distribucion_concat = df_distribucion_concat.reset_index(drop=True)

In [32]:
import datetime

## Se ha hecho una modificación manual en el primer registro en la hora de llegada en la hoja excel por inconcistencia en el tipo de dato 
df_distribucion_concat['salida_fecha_hora'] = pd.to_datetime(df_distribucion_concat['fecha_salida']).dt.date.astype(str) + ' ' + df_distribucion_concat['hora_salida'].astype(str)
df_distribucion_concat['llegada_fecha_hora'] = pd.to_datetime(df_distribucion_concat['fecha_llegada']).dt.date.astype(str) + ' ' + df_distribucion_concat['hora_llegada'].astype(str)

In [33]:
df_distribucion_concat.dtypes

no_serie                      object
conductor                     object
nombre_attsf                  object
fecha_salida          datetime64[ns]
hora_salida                   object
fecha_llegada         datetime64[ns]
hora_llegada                  object
km_salida                     object
km_llegada                    object
km_totales                    object
tm                            object
tipo_producto                 object
wilaya                        object
incidencias                   object
observaciones                float64
salida_fecha_hora             object
llegada_fecha_hora            object
dtype: object

In [34]:
df_distribucion_concat.loc[2817, 'llegada_fecha_hora'] = '2020-02-22 03:00:00'
df_distribucion_concat.loc[6390, 'llegada_fecha_hora'] = '2022-07-03 12:00:00'
df_distribucion_concat.loc[6396, 'llegada_fecha_hora'] = '2022-07-13 12:00:00'
df_distribucion_concat.loc[6779, 'llegada_fecha_hora'] = '2023-01-02 12:00:00'
df_distribucion_concat.loc[6796, 'llegada_fecha_hora'] = '2023-01-04 12:00:00'

In [35]:

df_distribucion_concat['llegada_fecha_hora'] = pd.to_datetime(df_distribucion_concat['llegada_fecha_hora'], format='%Y-%m-%d %H:%M:%S')

In [36]:
df_distribucion_concat.loc[170, 'salida_fecha_hora'] = '2019-04-04 08:00:00'
df_distribucion_concat.loc[171, 'salida_fecha_hora'] = '2019-04-04 08:00:00'
df_distribucion_concat.loc[169, 'salida_fecha_hora'] = '2019-04-04 08:00:00'
df_distribucion_concat.loc[653, 'salida_fecha_hora'] = '2019-04-04 08:00:00'

In [37]:
df_distribucion_concat.loc[169, 'salida_fecha_hora']

'2019-04-04 08:00:00'

In [38]:
df_distribucion_concat[df_distribucion_concat['salida_fecha_hora'] == '2019-04-04 8.00']

,no_serie,conductor,nombre_attsf,fecha_salida,hora_salida,fecha_llegada,hora_llegada,km_salida,km_llegada,km_totales,tm,tipo_producto,wilaya,incidencias,observaciones,salida_fecha_hora,llegada_fecha_hora


In [39]:
df_distribucion_concat['salida_fecha_hora'] = pd.to_datetime(df_distribucion_concat['salida_fecha_hora'], format='%Y-%m-%d %H:%M:%S')

In [40]:
df_distribucion_concat.dtypes

no_serie                      object
conductor                     object
nombre_attsf                  object
fecha_salida          datetime64[ns]
hora_salida                   object
fecha_llegada         datetime64[ns]
hora_llegada                  object
km_salida                     object
km_llegada                    object
km_totales                    object
tm                            object
tipo_producto                 object
wilaya                        object
incidencias                   object
observaciones                float64
salida_fecha_hora     datetime64[ns]
llegada_fecha_hora    datetime64[ns]
dtype: object

In [41]:
## Gestion de columnas de texto 
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.title()
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.lstrip()
df_distribucion_concat['conductor'] = df_distribucion_concat['conductor'].str.rstrip()

df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.title()
df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.lstrip()
df_distribucion_concat['wilaya'] = df_distribucion_concat['wilaya'].str.rstrip()

df_distribucion_concat['tipo_producto'] = df_distribucion_concat['tipo_producto'].str.upper()
df_distribucion_concat['tipo_producto'] = df_distribucion_concat['tipo_producto'].str.lstrip()
df_distribucion_concat['tipo_producto'] = df_distribucion_concat['tipo_producto'].str.rstrip()

d_unificados_wilaya = {'Aaiun': ['Aaiún', 'Aiun'], 
                       'Bojador' : ['Bujador', 'Bujdur']
                      } 

                      
## Bucle que unifica los datos en la columna wilaya 
for i in range(0, len(df_distribucion_concat['wilaya'])):
     for key in d_unificados_wilaya.keys():
         for value in d_unificados_wilaya[key]:
             if value in df_distribucion_concat.loc[i, 'wilaya']:
                 df_distribucion_concat.loc[i, 'wilaya'] = df_distribucion_concat.loc[i, 'wilaya'].replace(value, key)

In [42]:


## Bucle que unifica los datos de la columna conductor en el df_distribucion_concat 
for i in range(0, len(df_distribucion_concat['conductor'])):
     for key in d_unificados_conductor.keys():
         for value in d_unificados_conductor[key]:
             if value in df_distribucion_concat.loc[i, 'conductor']:
                 df_distribucion_concat.loc[i, 'conductor'] = df_distribucion_concat.loc[i, 'conductor'].replace(value, key)

df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(int)

In [43]:
## bucle que unifica nombres de camiones de tipo_vehiculo == CA 
for i in range (0, len(df_distribucion_concat['nombre_attsf'])):
    df_distribucion_concat.loc[i, 'nombre_attsf'] = 'CA' + str(df_distribucion_concat.loc[i, 'nombre_attsf'])

df_distribucion_concat

,no_serie,conductor,nombre_attsf,fecha_salida,hora_salida,fecha_llegada,hora_llegada,km_salida,km_llegada,km_totales,tm,tipo_producto,wilaya,incidencias,observaciones,salida_fecha_hora,llegada_fecha_hora
0,1,Seilum,CA10,2019-01-02,09:00:00,2019-01-02,15:42:00,NaN,NaN,106.0,11.85,CB,Aaiun,NaN,NaN,2019-01-02 09:00:00,2019-01-02 15:42:00
1,2,Said Al-Lal,CA14,2019-01-02,09:00:00,2019-01-02,16:38:00,NaN,NaN,106.0,6,CB,Aaiun,NaN,NaN,2019-01-02 09:00:00,2019-01-02 16:38:00
2,3,Said Nafi,CA17,2019-01-02,09:00:00,2019-01-02,18:10:00,NaN,NaN,106.0,5,CB,Aaiun,NaN,NaN,2019-01-02 09:00:00,2019-01-02 18:10:00
3,4,Limam Moh. Mehdi,CA20,2019-01-02,09:00:00,2019-01-02,17:17:00,NaN,NaN,106.0,12,CB,Aaiun,NaN,NaN,2019-01-02 09:00:00,2019-01-02 17:17:00
4,5,Embarec Ahmed,CA21,2019-01-02,09:00:00,2019-01-02,16:37:00,52957.0,53065.0,108.0,20,CB,Aaiun,NaN,NaN,2019-01-02 09:00:00,2019-01-02 16:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,195.0,Abdelmula,CA24,2023-01-30,08:00:00,2023-01-30,16:39:00,199812.0,200141.0,329.0,20.0,E,Dajla,NaN,NaN,2023-01-30 08:00:00,2023-01-30 16:39:00
6974,196.0,Salma Mujtar,CA25,2023-01-30,08:00:00,2023-01-30,16:30:00,135152.0,135184.0,32.0,20.0,E,Bojador,NaN,NaN,2023-01-30 08:00:00,2023-01-30 16:30:00
6975,197.0,Hasana Lehbib,CA26,2023-01-30,08:00:00,2023-01-30,17:00:00,228199.0,228271.0,72.0,20.0,E,Smara,NaN,NaN,2023-01-30 08:00:00,2023-01-30 17:00:00
6976,198.0,Bolla,CA11,2023-01-31,08:45:00,2023-01-31,16:00:00,NaN,NaN,77.0,10.0,CB,Smara,NaN,NaN,2023-01-31 08:45:00,2023-01-31 16:00:00


In [44]:
## merge de la tabla distribuciones con la tabla tipo_producto 
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_tipo_producto, how='outer', left_on='tipo_producto', 
right_on='tipo_producto')

## merge de la tabla distribuciones con la tabla conductor
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_conductor, how='outer')

#df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(int)
df_distribucion_concat['nombre_attsf'] = df_distribucion_concat['nombre_attsf'].astype(str)

## merge de la tabla distribuciones con la tabla camion
## hay problema con el tipo de datos de columna camion de df_distribucion_concat 
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_camion[['id_camion', 'nombre_attsf']], how='left', left_on='nombre_attsf', right_on='nombre_attsf')

## merge de la tabla distribuciones con la tabla wilaya
df_distribucion_concat = pd.merge(left = df_distribucion_concat, right = df_wilaya[['id_wilaya', 'wilaya']], how='inner')

In [45]:
## reconstrucción de información histórica distribución

# se arma de nuevo el dataframe distribuciones 
d_distribucion = {'id_conductor': df_distribucion_concat['id_conductor'], 'id_tipo_producto': df_distribucion_concat['id_tipo_producto'], 'id_camion': df_distribucion_concat['id_camion'],  
    'id_wilaya': df_distribucion_concat['id_wilaya'], 'no_serie': df_distribucion_concat['no_serie'],
    'salida_fecha_hora': df_distribucion_concat['salida_fecha_hora'], 'llegada_fecha_hora': df_distribucion_concat['llegada_fecha_hora'], 
    'km_salida': df_distribucion_concat['km_salida'], 'km_llegada': df_distribucion_concat['km_llegada'], 'km_totales': df_distribucion_concat['km_totales'], 'tm': df_distribucion_concat['tm'], 
    'incidencias': df_distribucion_concat['incidencias'], 'observaciones': df_distribucion_concat['observaciones']}
df_distribucion= pd.DataFrame(d_distribucion) 


## ordenar valores por fecha 
df_distribucion = df_distribucion.sort_values('salida_fecha_hora')


## resetear index 
df_distribucion = df_distribucion.reset_index(drop=True)
df_distribucion.index +=1 
df_distribucion = df_distribucion.reset_index(names='id_distribucion')

## para este nùmero de serie de determina que el dato faltante corresponde a 822 segun el consecutivo de datos de origen 
df_distribucion.loc[df_distribucion[df_distribucion['no_serie'] == ' '].index, 'no_serie'] = 822

df_distribucion['no_serie'] = df_distribucion['no_serie'].astype(int)
# comprobar que no hay valores nulos y sacar su indice 
#df_distribucion[pd.isna(df_distribucion['id_tipo_producto']) == True]
df_distribucion['id_tipo_producto'] = df_distribucion['id_tipo_producto'].astype(int)
df_distribucion['id_wilaya'] = df_distribucion['id_wilaya'].astype(int)
df_distribucion['id_camion'] = df_distribucion['id_camion'].astype(int)
df_distribucion['id_conductor'] = df_distribucion['id_conductor'].astype(int) 


In [372]:
#df_distribucion[df_distribucion['llegada_fecha_hora'].str.len() != 19]

In [46]:
df_distribucion.loc[6791, 'llegada_fecha_hora']

Timestamp('2023-01-04 12:00:00')

In [262]:
# df_distribucion.loc[2818, 'llegada_fecha_hora'] = '2020-02-22 03:00:00'
# df_distribucion.loc[6387, 'llegada_fecha_hora'] = '2022-07-03 12:00:00'
# df_distribucion.loc[6401, 'llegada_fecha_hora'] = '2022-07-13 12:00:00'
# df_distribucion.loc[6782, 'llegada_fecha_hora'] = '2023-01-02 12:00:00'
# df_distribucion.loc[6791, 'llegada_fecha_hora'] = '2023-01-04 12:00:00'

In [387]:
#df_distribucion[df_distribucion['salida_fecha_hora'].str.len() != 19]

In [264]:
#df_distribucion.loc[658, 'salida_fecha_hora'] = '2019-04-04 08:00:00'

In [47]:
df_distribucion.loc[df_distribucion[df_distribucion['km_salida'] == ' '].index, 'km_salida'] = float('nan')

In [48]:
df_distribucion[df_distribucion['tm'] == '4,,00']

,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
736,737,2,1,80,3,741,2019-04-14 08:00:00,2019-04-14 15:30:00,NaN,NaN,30.0,"4,,00",NaN,NaN


In [49]:
df_distribucion.loc[df_distribucion[df_distribucion['tm'] == '4,,00'].index, 'tm'] = 4.0

In [50]:
# Seleccionar las filas que no tienen valores de tipo float en la columna 'km_totales'
indices_no_float = df_distribucion.loc[df_distribucion['km_totales'].apply(lambda x: not isinstance(x, float))].index

# Imprimir los índices de las filas que no tienen valores de tipo float
print("Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son: ", indices_no_float)


Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son:  Int64Index([5446, 5447, 5448, 5449, 5450, 5451, 5452, 5453, 5454, 5455,
            ...
            6769, 6770, 6771, 6772, 6773, 6774, 6775, 6776, 6777, 6778],
           dtype='int64', length=1333)


In [587]:
#df_distribucion.loc[indices_no_float, 'km_totales'] = df_distribucion.loc[indices_no_float, 'km_totales'].astype(float)

In [51]:
df_distribucion[df_distribucion['km_totales'] == '77km']

,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
6481,6482,23,1,75,5,1029,2022-07-27 08:00:00,2022-07-27 13:00:00,NaN,NaN,77km,3,NaN,NaN


In [52]:
df_distribucion.loc[df_distribucion[df_distribucion['km_totales'] == '77km'].index, 'km_totales'] = 77 

In [53]:
df_distribucion[df_distribucion['km_totales'] == '30km']

,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
6485,6486,20,1,82,3,1032,2022-07-27 08:00:00,2022-07-27 14:00:00,NaN,NaN,30km,1,NaN,NaN


In [54]:
df_distribucion.loc[df_distribucion[df_distribucion['km_totales'] == '30km'].index, 'km_totales'] = 30

In [55]:
# Seleccionar las filas que no tienen valores de tipo float en la columna 'km_totales'
indices_no_float = df_distribucion.loc[df_distribucion['km_salida'].apply(lambda x: not isinstance(x, float))].index

# Imprimir los índices de las filas que no tienen valores de tipo float
print("Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son: ", indices_no_float)

Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son:  Int64Index([2474, 2477, 2482, 2483, 2486, 2493, 2495, 2496, 2501, 2503,
            ...
            6741, 6746, 6748, 6749, 6757, 6761, 6763, 6765, 6776, 6778],
           dtype='int64', length=1098)


In [595]:
#df_distribucion.loc[indices_no_float, 'km_salida'] = df_distribucion.loc[indices_no_float, 'km_salida'].astype(float)

In [56]:
df_distribucion[df_distribucion['km_salida'] == '                                                                                                                                                                 ']

,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
6006,6007,20,2,82,3,552,2022-04-13 09:00:00,2022-04-13 17:30:00,...,NaN,106,10,NaN,NaN


In [57]:
indice_1 = df_distribucion[df_distribucion['km_salida'] == '                                                                                                                                                                 '].index
df_distribucion.loc[indice_1, 'km_salida'] = float('nan')

In [58]:
df_distribucion[df_distribucion['km_salida'] == '                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               ']


,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones
3884,3885,18,1,81,5,1417,2020-07-23 06:30:00,2020-07-23 18:20:00,...,NaN,77.0,10.0,NaN,NaN


In [59]:
indice_2 = df_distribucion[df_distribucion['km_salida'] == '                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               '].index
df_distribucion.loc[indice_2, 'km_salida'] = float('nan')

In [60]:
# Seleccionar las filas que no tienen valores de tipo float en la columna 'km_totales'
indices_no_float = df_distribucion.loc[df_distribucion['km_llegada'].apply(lambda x: not isinstance(x, float))].index

# Imprimir los índices de las filas que no tienen valores de tipo float
print("Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son: ", indices_no_float)

Los índices de las filas que no tienen valores de tipo float en la columna 'km_totales' son:  Int64Index([4876, 4881, 4884, 4886, 4895, 4897, 4905, 4907, 4910, 4927,
            ...
            5422, 5426, 5428, 5431, 5435, 5436, 5441, 5442, 5443, 5444],
           dtype='int64', length=164)


In [601]:
#df_distribucion.loc[indices_no_float, 'km_llegada'] = df_distribucion.loc[indices_no_float, 'km_llegada'].astype(float)

In [61]:
df_distribucion[df_distribucion['km_llegada'] == '                                                                                                                                                                 ']

,id_distribucion,id_conductor,id_tipo_producto,id_camion,id_wilaya,no_serie,salida_fecha_hora,llegada_fecha_hora,km_salida,km_llegada,km_totales,tm,incidencias,observaciones


In [62]:
df_distribucion['km_llegada'] = pd.to_numeric(df_distribucion['km_llegada'], errors='coerce')

In [63]:
df_distribucion['km_salida'] = pd.to_numeric(df_distribucion['km_salida'], errors='coerce')

In [64]:
df_distribucion['km_totales'] = pd.to_numeric(df_distribucion['km_totales'], errors='coerce')

In [65]:
df_distribucion['tm'] = pd.to_numeric(df_distribucion['tm'], errors='coerce')

In [72]:
df_distribucion['observaciones'] = df_distribucion['observaciones'].astype(str)

In [77]:
df_distribucion.to_csv(path_output / 'distribucion.csv', index=False, encoding = 'latin1')

In [73]:
df_distribucion.dtypes

id_distribucion                int64
id_conductor                   int32
id_tipo_producto               int32
id_camion                      int32
id_wilaya                      int32
no_serie                       int32
salida_fecha_hora     datetime64[ns]
llegada_fecha_hora    datetime64[ns]
km_salida                    float64
km_llegada                   float64
km_totales                   float64
tm                           float64
incidencias                   object
observaciones                 object
dtype: object

Código para gestionar los duplicados 

In [75]:
df_distribucion['observaciones']

0       nan
1       nan
2       nan
3       nan
4       nan
       ... 
6973    nan
6974    nan
6975    nan
6976    nan
6977    nan
Name: observaciones, Length: 6978, dtype: object

In [ ]:
camiones = df_camion_all['nombre_attsf']
wilayas = df_camion_all['wilaya']

for camion, wilaya in zip(camiones, wilayas): 
    indice = df_camion_all[(df_camion_all['nombre_attsf'] == camion) & (df_camion_all['wilaya'] == wilaya)].index
    if len(indice) == 2:
        df_camion_all.loc[indice[0]] = df_camion_all.loc[indice[0]].fillna(df_camion_all.loc[indice[1]])
        df_camion_all.loc[indice[1]] = df_camion_all.loc[indice[1]].fillna(df_camion_all.loc[indice[0]])
        if df_camion_all.loc[indice[0], 'marca'] == 'Volvo': 
            df_camion_all.loc[indice[0], 'marca'] = df_camion_all.loc[indice[1], 'marca']
        if df_camion_all.loc[indice[1], 'marca'] == 'Volvo': 
            df_camion_all.loc[indice[1], 'marca'] = df_camion_all.loc[indice[0], 'marca']